In [217]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv('naukri_com-job_sample.csv')
df.head()

,company,education,experience,industry,jobdescription,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,skills,uniq_id
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,210516002263,Chennai,Walkin Data Entry Operator (night Shift),NaN,"1,50,000 - 2,25,000 P.A",2016-05-21 19:30:00 +0000,NaN,ITES,43b19632647068535437c774b6ca6cf8
1,find live infotech,UG: B.Tech/B.E. - Any Specialization PG:MBA/PG...,0 - 0 yrs,Advertising / PR / MR / Event Management,Job Description Send me Jobs like this Quali...,210516002391,Chennai,Work Based Onhome Based Part Time.,60.0,"1,50,000 - 2,50,000 P.A. 20000",2016-05-21 19:30:00 +0000,NaN,Marketing,d4c72325e57f89f364812b5ed5a795f0
2,Softtech Career Infosystem Pvt. Ltd,UG: Any Graduate - Any Specialization PG:Any P...,4 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this - as ...,101016900534,Bengaluru,Pl/sql Developer - SQL,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,c47df6f4cfdf5b46f1fd713ba61b9eba
3,Onboard HRServices LLP,UG: Any Graduate - Any Specialization PG:CA Do...,11 - 15 yrs,Banking / Financial Services / Broking,Job Description Send me Jobs like this - Inv...,81016900536,"Mumbai, Bengaluru, Kolkata, Chennai, Coimbator...",Manager/ad/partner - Indirect Tax - CA,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,Accounts,115d28f140f694dd1cc61c53d03c66ae
4,Spire Technologies and Solutions Pvt. Ltd.,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,6 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this Pleas...,120916002122,Bengaluru,JAVA Technical Lead (6-8 yrs) -,4.0,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,a12553fc03bc7bcced8b1bb8963f97b4


In [218]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [219]:
y = df['payrate']
X = df.drop(['payrate'], axis=1)

In [220]:
df.drop(df[y.isnull()].index, inplace=True)

In [221]:
y = df['payrate']
yenc = LabelEncoder()
y =  yenc.fit_transform(y)
X = df.drop(['payrate','uniq_id','jobid','skills','jobdescription','site_name','postdate','numberofpositions'], axis=1)

In [222]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21903 entries, 0 to 21999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   company              21903 non-null  object
 1   education            19969 non-null  object
 2   experience           21903 non-null  object
 3   industry             21903 non-null  object
 4   joblocation_address  21406 non-null  object
 5   jobtitle             21903 non-null  object
dtypes: object(6)
memory usage: 1.2+ MB


In [223]:
cat_cols = X.select_dtypes(include=['object']).columns
cat_cols

Index(['company', 'education', 'experience', 'industry', 'joblocation_address',
       'jobtitle'],
      dtype='object')

In [224]:
num_cols = X.select_dtypes(include=['int64','float64']).columns
num_cols

Index([], dtype='object')

In [225]:
numpipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
catpipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder())
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numpipe, num_cols),
    ('cat', catpipe, cat_cols)
])

In [226]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index([], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinal',
                                                  OrdinalEncoder())]),
                                 Index(['company', 'education', 'experience', 'industry', 'joblocation_address',
       'jobtitle'],
      dtype='object'))])

In [227]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeClassifier())
])
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index([], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  Index(['company', 'education', 'experience', 'industry', 'joblocation_address',
       'jobtitle'],
      dtype='object'))])),
                ('regressor', DecisionTreeClassifier())])

In [228]:
model.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index([], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  Index(['company', 'education', 'experience', 'industry', 'joblocation_address',
       'jobtitle'],
      dtype='object'))])),
                ('regressor', DecisionTreeClassifier())])

In [229]:
model.score(X, y)

0.9997717207688445

In [230]:
ypred = model.predict(X)

In [231]:
import seaborn as sns
import matplotlib.pyplot as plt

print(classification_report(y, ypred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00        15
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00        21
          13       1.00      1.00      1.00        15
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         1
          16       1.00      1.00      1.00         1
          17       1.00    

In [232]:
import joblib

joblib.dump(model, 'model.joblib')

['model.joblib']

In [233]:
def load_model():
    return joblib.load('model.joblib')

def predict(data):
    model = load_model()
    return model.predict(data)[0]

In [234]:
X.head(1)

,company,education,experience,industry,joblocation_address,jobtitle
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Chennai,Walkin Data Entry Operator (night Shift)


In [235]:
Xunique = []
for col in X.columns:
    if X[col].dtype == 'object':
        Xunique.append(X[col].unique())
Xunique


[array(['MM Media Pvt Ltd', 'find live infotech',
        'Softtech Career Infosystem Pvt. Ltd', ...,
        'TeamLease Services Limited hiring for BPA Solution Architect',
        'Austere Technology Solutions', 'Musk - Startup'], dtype=object),
 array(['UG: B.Tech/B.E. - Any Specialization PG:Any Postgraduate - Any Specialization, Post Graduation Not Required',
        'UG: B.Tech/B.E. - Any Specialization PG:MBA/PGDM - Any Specialization',
        'UG: Any Graduate - Any Specialization PG:Any Postgraduate Doctorate:Doctorate Not Required',
        ..., 'PG:MS/M.Sc(Science) - Any Specialization, Statistics',
        'UG: B.Tech/B.E. - Any Specialization PG:MCA - Computers, M.Tech - Any Specialization, MS/M.Sc(Science) - Any Specialization Doctorate:Doctorate Not Required',
        'UG: B.Tech/B.E. - Computers PG:MS/M.Sc(Science) - Computers Doctorate:Doctorate Not Required'],
       dtype=object),
 array(['0 - 1 yrs', '0 - 0 yrs', '4 - 8 yrs', '11 - 15 yrs', '6 - 8 yrs',
        '2 

In [236]:
joblib.dump(Xunique, 'Xunique.joblib')
joblib.dump(yenc, 'yenc.joblib')

['yenc.joblib']